# Sentinel-1 GeoMAD

This notebook runs a process to load Sentinel-1 data and produce
a geometric median and median absolute deviations. Two test regions
are currently available, one in Tasmania and one in Moretons Bay.

Key requirements include Datacube and a specific branch of
[odc-algo](https://github.com/opendatacube/odc-algo/tree/add-rust-geomedian-impl).

In [ ]:
import odc.geo.xr  # noqa: F401
from datacube import Datacube
from distributed import Client
from dask import config
from numpy import log10

from odc.algo import geomedian_with_mads
from odc.stac import configure_rio

from collections import namedtuple

# from easi_tools.notebook_utils import localcluster_dashboard
# from easi_tools import EasiDefaults

In [ ]:
# Connect to the datacube and set up some config
dc = Datacube()

# Rasterio defaults
configure_rio(cloud_defaults=True)

# Easi defaults
# easi = EasiDefaults()

# A simple data structure for our study sites
study_site = namedtuple("study_site", ["name", "bbox"])

In [ ]:
# Configure Dask using defaults
client = Client()

# print(localcluster_dashboard(client=client, server=easi.hub))

In [ ]:
# Change process parameters here

# Study site in Tasmania
site = study_site("southern_tasmania", [146.2357, -43.6796, 147.147, -42.9305])

# Study site in Moreton Bay
# site = study_site("moreton_bay", [152.00, -27.50, 153.20, -26.50])

year = "2023"

data = dc.load(
    product="sentinel1_grd_gamma0_beta",
    lon=(site.bbox[0], site.bbox[2]),
    lat=(site.bbox[1], site.bbox[3]),
    time=year,
    resolution=(-20, 20),
    output_crs="EPSG:3577",
    dask_chunks=dict(x=2048, y=2048),
    group_by="solar_day",
    measurements=["vv", "vh"],
)

# Mask out nodata values
data = data.where(data.vv > 0)

data

In [ ]:
# Get the geomedian and mad bands
geomad = geomedian_with_mads(data, work_chunks=(2048, 2048), num_threads=32)

# Calculate means
for band in ["vv", "vh"]:
    geomad[f"{band}_mean"] = data[band].mean("time")

geomad

In [ ]:
# This step run the computation on Dask. Open the dashboard
# link above to watch it process.
computed = geomad.compute()

In [ ]:
computed["vv"].plot.imshow(robust=True)

In [ ]:
computed["vv_mean"].plot.imshow(robust=True)

In [ ]:
(computed["vv"] - computed["vv_mean"]).plot.imshow(robust=True)

In [ ]:
computed["vv_log"] = 10 * log10(computed.vv)
computed["vh_log"] = 10 * log10(computed.vh)
computed["vh_vv_log"] = 0.5 * (computed.vh_log / computed.vv_log)

computed.odc.explore(bands=["vv_log", "vh_log", "vh_vv_log"], vmin=-30, vmax=0)